
### This Interactive Notebook was generated by ML.NET Tooling.

The code below demonstrates how to

1. Define the model input and output schema
1. Load in data from a text file to an IDataView
1. Set up the training pipeline with data transforms
1. Choose an algorithm and append it to the pipeline
1. Train the model
1. Evaluate the model
1. Consume the model


## Install the necessary NuGet packages for training ML.NET model and plotting:

In [1]:

/* ML.NET Model Builder generated Notebook file. Notebook files contain both code snippets and rich text elements.
Use the "run" button in the left margin to execute each code snippet and explore ML.NET. */

#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 

#r "nuget:Microsoft.ML,1.7.0"
#r "nuget:Microsoft.Data.Analysis,0.19.0"
#r "nuget:XPlot.Plotly.Interactive, 4.0.1"


Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.0 Microsoft.ML, 1.7.0 XPlot.Plotly.Interactive, 4.0.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `XPlot.Plotly.Interactive.dll`

: Microsoft.DotNet.Interactive.KernelExtensionLoadException: Failure loading Kernel Extension
 ---> System.MissingMethodException: Method not found: 'Microsoft.DotNet.Interactive.DisplayedValue Microsoft.DotNet.Interactive.KernelInvocationContextExtensions.Display(Microsoft.DotNet.Interactive.KernelInvocationContext, System.Object, System.String)'.
   at XPlot.Plotly.Interactive.KernelExtension.Microsoft.DotNet.Interactive.IKernelExtension.OnLoadAsync(Kernel kernel)
   at Microsoft.DotNet.Interactive.Extensions.AssemblyBasedExtensionLoader.LoadFromAssembly(FileInfo assemblyFile, Kernel kernel, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Extensions\AssemblyBasedExtensionLoader.cs:line 132
   --- End of inner exception stack trace ---

In [1]:

// Import common usings.
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

## Define the model input and output schemas:

In [1]:
﻿// Define the model input schema (which columns you will be loading in for training)
public class ModelInput
{
    [ColumnName(@"AnualCompensation"), LoadColumn(0)]
    public float AnualCompensation { get; set; }
    
    [ColumnName(@"Company"), LoadColumn(1)]
    public string Company { get; set; }
    
    [ColumnName(@"Title"), LoadColumn(2)]
    public string Title { get; set; }
    
    [ColumnName(@"YearsExperience"), LoadColumn(3)]
    public float YearsExperience { get; set; }
    
    [ColumnName(@"YearsCompany"), LoadColumn(4)]
    public float YearsCompany { get; set; }
    
    [ColumnName(@"Gender"), LoadColumn(5)]
    public string Gender { get; set; }
    
    [ColumnName(@"Location"), LoadColumn(6)]
    public string Location { get; set; }
    
    [ColumnName(@"Race"), LoadColumn(7)]
    public string Race { get; set; }
    
    [ColumnName(@"Education"), LoadColumn(8)]
    public string Education { get; set; }
    
}


// Define the model output schema (what the model will return)
public class ModelOutput
{
    public float Score { get; set; }
}



## Create MLContext and load training data:

In [1]:
// Create a new MLContext (the starting point for all ML.NET operations)
var mlContext = new MLContext();

// Define path to training data
string trainDataPath = @"F:\DATASETS ML\DataSTEMSalary.csv";

// Load data from a text file to an IDataView (a flexible, efficient way of describing tabular data)
IDataView trainData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: trainDataPath ,
    hasHeader: true ,
    separatorChar: ',',
    allowQuoting: true,
    allowSparse: false);

// Display training data schema
display(trainData.Schema); 



index Name Index IsHidden Type Annotations 0 AnualCompensation 0 False RawType System.Single Schema [ ] 1 Company 1 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 2 Title 2 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 3 YearsExperience 3 False RawType System.Single Schema [ ] 4 YearsCompany 4 False RawType System.Single Schema [ ] 5 Gender 5 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 6 Location 6 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 7 Race 7 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 8 Education 8 False RawType System.ReadOnlyMemory<System.Char> Schema [ ]

In [1]:
display(h4("Showing 5 rows from training DataView:"));
var fewRows = mlContext.Data.CreateEnumerable<ModelInput>(trainData, reuseRowObject: false)
                    .Take(5)
                    .ToList();
display(fewRows);

Showing 5 rows from training DataView:

index,AnualCompensation,Company,Title,YearsExperience,YearsCompany,Gender,Location,Race,Education
0,127000,Oracle,Product Manager,1.5,1.5,NA,"Redwood City, CA",NA,NA
1,100000,eBay,Software Engineer,5,3,NA,"San Francisco, CA",NA,NA
2,310000,Amazon,Product Manager,8,0,NA,"Seattle, WA",NA,NA
3,372000,Apple,Software Engineering Manager,7,5,NA,"Sunnyvale, CA",NA,NA
4,157000,Microsoft,Software Engineer,5,3,NA,"Mountain View, CA",NA,NA


## Create the training pipeline, choose an algorithm, and train the model:

In [1]:
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML;
// Append the trainer to the data processing pipeline
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding(new []{new InputOutputColumnPair(@"Title", @"Title"),new InputOutputColumnPair(@"Gender", @"Gender"),new InputOutputColumnPair(@"Race", @"Race"),new InputOutputColumnPair(@"Education", @"Education")})      
                 .Append(mlContext.Transforms.ReplaceMissingValues(new []{new InputOutputColumnPair(@"YearsExperience", @"YearsExperience"),new InputOutputColumnPair(@"YearsCompany", @"YearsCompany")}))      
                 .Append(mlContext.Transforms.Text.FeaturizeText(inputColumnName:@"Company",outputColumnName:@"Company"))      
                 .Append(mlContext.Transforms.Text.FeaturizeText(inputColumnName:@"Location",outputColumnName:@"Location"))      
                 .Append(mlContext.Transforms.Concatenate(@"Features", new []{@"Title",@"Gender",@"Race",@"Education",@"YearsExperience",@"YearsCompany",@"Company",@"Location"}))      
                 .Append(mlContext.Transforms.NormalizeMinMax(@"Features", @"Features"))      
                 .Append(mlContext.Regression.Trainers.LbfgsPoissonRegression(new LbfgsPoissonRegressionTrainer.Options(){L1Regularization=0.03125F,L2Regularization=0.03125F,LabelColumnName=@"AnualCompensation",FeatureColumnName=@"Features"}));

// Train the model (fit the model to the training data)
var model = pipeline.Fit(trainData);



## Consume the model

In [1]:
﻿ // Define sample model input
var sampleData = new ModelInput()
{
    Company = @"Avanade",
    Title = @"Software Engineer",
    YearsExperience = 3F,
    YearsCompany = 1F,
    Gender = @"NA",
    Location = @"San Francisco, CA",
    Race = @"White",
    Education = @"NA",
};

// Create a Prediction Engine (used to make single predictions)
var predEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
// Use the model and Prediction Engine to predict on new sample data
var predictionResult = predEngine.Predict(sampleData);
Console.WriteLine("Using model to make single prediction -- Comparing actual AnualCompensation with predicted AnualCompensation from sample data...\n\n");

Console.WriteLine($"AnualCompensation: {100000F}");
Console.WriteLine($"Company: {@"eBay"}");
Console.WriteLine($"Title: {@"Software Engineer"}");
Console.WriteLine($"YearsExperience: {5F}");
Console.WriteLine($"YearsCompany: {3F}");
Console.WriteLine($"Gender: {@"NA"}");
Console.WriteLine($"Location: {@"San Francisco, CA"}");
Console.WriteLine($"Race: {@"NA"}");
Console.WriteLine($"Education: {@"NA"}");

Console.WriteLine($"\n\nPredicted AnualCompensation: {predictionResult.Score}\n\n");




Predicted AnualCompensation: 114131.58




## Evaluate the model:

In [1]:
// Evaluate the model using the cross validation method
// Learn more about cross validation at https://aka.ms/mlnet-cross-validation

var crossValidationResults = mlContext.Regression.CrossValidate(trainData, pipeline, numberOfFolds: 5, labelColumnName:"AnualCompensation");

// Define which model evaluation metrics you'd like to see
var L1 = crossValidationResults.Select(r => r.Metrics.MeanAbsoluteError);
var L2 = crossValidationResults.Select(r => r.Metrics.MeanSquaredError);
var RMS = crossValidationResults.Select(r => r.Metrics.RootMeanSquaredError);
var lossFunction = crossValidationResults.Select(r => r.Metrics.LossFunction);
var R2 = crossValidationResults.Select(r => r.Metrics.RSquared);

// Print out the evaluation metrics
var metricNames = new StringDataFrameColumn("Metric Name", new[] {"Average L1 Loss", "Average L2 Loss", "Average RMS", "Average Loss Function", "Average R-Squared"});
var metricValues = new StringDataFrameColumn("Value",new[] {$"{L1.Average():0.###}", $"{L2.Average():0.###}", $"{RMS.Average():0.###}", $"{lossFunction.Average():0.###}", $"{R2.Average():0.###}"});
var stats = new DataFrame(metricNames, metricValues);

stats



index,Metric Name,Value
0,Average L1 Loss,49712.666
1,Average L2 Loss,8076337396.084
2,Average RMS,89738.362
3,Average Loss Function,8076337424.822
4,Average R-Squared,0.577


### This Interactive Notebook was generated by ML.NET Tooling.

The code below demonstrates several methods to explain your model, including how to get and display

1. A Histogram of the distribution of number of instances
1. A Scatter Plot
1. Compare actual values to predicted values in a scatter plot
1. The importance of different features

In [1]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.IO;
using System.Linq;
using XPlot.Plotly;

## Compare Distribution of Number of Instances

In [1]:
// Extract some data into arrays for plotting

int numberOfRows = 5000;

// Columns was determined by inputted data
float[] AnualCompensation = trainData.GetColumn<float>("AnualCompensation").Take(numberOfRows).ToArray();

// Distribution of Number of Instances
var histogram = Chart.Plot(new Histogram(){x = AnualCompensation, autobinx = false, nbinsx = 20});
var layout = new Layout.Layout(){title="AnualCompensation vs Number of Instances"};
histogram.WithLayout(layout);
histogram.WithXTitle("AnualCompensation");
histogram.WithYTitle("Number of Instances");

display(histogram);



Height,Id,PlotlySrc,Width
500,c6ad20a5-6626-44e6-a13e-4e57447011fd,https://cdn.plot.ly/plotly-latest.min.js,900


In [1]:
int numberOfRows = 2000;

float[] AnualCompensation = trainData.GetColumn<float>("AnualCompensation").Take(numberOfRows).ToArray();
float[] AnualCompensation = trainData.GetColumn<float>("AnualCompensation").Take(numberOfRows).ToArray();


var chartFareVsPassengers = Chart.Plot(
    new Scatter()
    {
        x = AnualCompensation,
        y = AnualCompensation,
        mode = "markers",
    }
);

var layout = new Layout.Layout(){title="Plot AnualCompensation depending on AnualCompensation"};
chartFareVsPassengers.WithLayout(layout);
chartFareVsPassengers.Width = 500;
chartFareVsPassengers.Height = 500;
chartFareVsPassengers.WithXTitle("AnualCompensation");
chartFareVsPassengers.WithYTitle("AnualCompensation");
chartFareVsPassengers.WithLegend(false);

display(chartFareVsPassengers);



: (10,13): error CS0229: Ambiguity between 'AnualCompensation' and 'AnualCompensation'
(11,13): error CS0229: Ambiguity between 'AnualCompensation' and 'AnualCompensation'
(4,9): error CS0102: The type 'Submission#20' already contains a definition for 'AnualCompensation'

## Compare actual values to predicted values in a scatter plot

In [1]:
// Use the model to make batch predictions on training data
var testResults = model.Transform(trainData);

// Get the actual values from the dataset
var trueValues = testResults.GetColumn<float>("AnualCompensation");

// Get the predicted values from the test results
var predictedValues = testResults.GetColumn<float>("Score");

// Create scatter plot of actual vs predicted values
var predictedVsTrue = new Scattergl()
{
    x = trueValues,
    y = predictedValues,
    mode = "markers",
};

var maximumValue = Math.Max(trueValues.Max(), predictedValues.Max());

var perfectLine = new Scattergl()
{
    x = new[] {0, maximumValue},
    y = new[] {0, maximumValue},
    mode = "lines",
};

var chart = Chart.Plot(new[] {predictedVsTrue, perfectLine });
chart.WithXTitle("Actual Values");
chart.WithYTitle("Predicted Values");
chart.WithLegend(false);
chart.Width = 600;
chart.Height = 600;
display(chart);



## Calculate and graph the Permutation Feature Importance (PFI)

In [1]:
// Calculate PFI
var predictor = (ISingleFeaturePredictionTransformer<object>) ((IEnumerable<ITransformer>)model).Last();
var preprocessedTrainData = model.Transform(trainData);


VBuffer<ReadOnlyMemory<char>> nameBuffer = default;
preprocessedTrainData.Schema["Features"].Annotations.GetValue("SlotNames", ref nameBuffer); // NOTE: The column name "Features" needs to match the featureColumnName used in the trainer, the name "SlotNames" is always the same regardless of trainer.
var featureColumnNames = nameBuffer.DenseValues().ToList();

ImmutableArray<RegressionMetricsStatistics> permutationFeatureImportance =
    mlContext.Regression
    .PermutationFeatureImportance(predictor, preprocessedTrainData, permutationCount: 1, labelColumnName: "AnualCompensation");

var featureImportanceMetrics =
     permutationFeatureImportance
     .Select((metric, index) => new { index, metric.RSquared })
     .OrderByDescending(myFeatures => Math.Abs(myFeatures.RSquared.Mean));

    
var featureNames = new List<string>();
var featurePFI = new List<double>();
foreach (var feature in featureImportanceMetrics)
{
     featureNames.Add($"{featureColumnNames[feature.index],-20}");
     featurePFI.Add(feature.RSquared.Mean);
}
var featureImportance = new DataFrame(new StringDataFrameColumn("Feature", featureNames.ToArray() ), new DoubleDataFrameColumn("R-Squared Impact",featurePFI.ToArray()));
    
featureImportance



index,Feature,R-Squared Impact
0,Gender.Male,-7.056607622274875
1,Gender.NA,-6.610475187306693
2,Gender.Female,-2.4264383849013837
3,Education.NA,-1.7718823421066603
4,Race.NA,-1.7584853292969036
5,Education.Master's Degree,-1.47739352329447
6,Race.Asian,-1.3747586515444241
7,Education.Bachelor's Degree,-1.1630033340083634
8,Race.White,-1.0182921237342641
9,Education.PhD,-0.3806474286340624


In [1]:
// Graph the PFI results
var pfiBar = new Bar()
{
    x = featureNames,
    y = featurePFI,
    dy = featurePFI[0]/100
};

var pfiChart = Chart.Plot(pfiBar);
pfiChart.WithXTitle("Feature");
pfiChart.WithYTitle("Contribution (delta R-Squared)");
pfiChart.Width = 600;
pfiChart.Height = 600;
display(pfiChart);

Height,Id,PlotlySrc,Width
600,4c24c7e6-bb24-4574-bdb0-9c27d52b7a22,https://cdn.plot.ly/plotly-latest.min.js,600
